In [ ]:
def ADC_slice(bvalues, slicedata):
    min_adc = 0
    max_adc = 3.0
    eps = 1e-7
    numrows, numcols, numbvalues = slicedata.shape
    adc_map = np.zeros((numrows, numcols))
    for row in range(numrows):
        for col in range(numcols):
            ydata = np.squeeze(slicedata[row,col,:])
            adc = np.polyfit(bvalues.flatten()/1000, np.log(ydata + eps), 1)
            adc = -adc[0]
            adc_map[row, col] =  max(min(adc, max_adc), min_adc)
    return adc_map

In [ ]:
def detect_PIDS_slice(b, S):
    """ Inputs: b - diffusion weight values used in image
                S - Hybrid Multi-dimensional image
        Outputs:
                PIDS_ADC1 : Binary Map with voxels ADC > 3 (could mean motion induced signal loss at high-b)
                PIDS_ADC2 : Binary Map with voxels ADC < 0 (could mean the voxel is below the noise level)
                PIDS_b_decay : Binary Map with voxels disobeying decay rule along b direction
                PIDS_TE_decay : Binary Map with voxels disobeying decay rule along TE direction
    """
    
    eps = 1e-7
    num_rows, num_cols, num_bvalues, num_TEs = S.shape
    PIDS_ADC1 = np.zeros((num_rows, num_cols))
    PIDS_ADC2 = np.zeros((num_rows, num_cols))
    PIDS_b_decay = np.zeros((num_rows, num_cols, num_bvalues))
    PIDS_TE_decay = np.zeros((num_rows, num_cols, num_TEs))
    for row in range(num_rows):
         for col in range(num_cols):
            for _b in range(num_bvalues):
                ydata = np.squeeze(S[row,col, :, _b])
                if not _b:
                    adc = np.polyfit(b.flatten()/1000, np.log(ydata + eps), 1)
                    adc = -adc[0]    
                    PIDS_ADC1[row, col] = int(adc > 3)
                    PIDS_ADC2[row, col] = int(adc < 0)
                to_compare = ydata.copy()
                to_compare[1:] = ydata[:3]
                is_pids = ydata - to_compare
                is_pids = sum(is_pids > 0) >0
                PIDS_b_decay[row, col, _b] = is_pids
            for _te in range(num_TEs):
                ydata = np.squeeze(S[row,col, _te, :])
                to_compare = ydata.copy()
                to_compare[1:] = ydata[:3]
                is_pids = ydata - to_compare
                is_pids = sum(is_pids > 0) >0
                PIDS_TE_decay[row, col, _te] = is_pids
    return PIDS_ADC1, PIDS_ADC2, PIDS_b_decay, PIDS_TE_decay

In [ ]:
adc_map = ADC_slice(b, hybrid_crop[:,:, sliceIndex, :, 0])
PIDS_1, PIDS_2, PIDS_3, PIDS_4 = detect_PIDS_slice(b, hybrid_crop[:,:, sliceIndex, :, :])
fig, ax = plt.subplots(1,3, figsize = (30,10))
ax[0].imshow(adc_map, cmap='gray')
ax[0].set_title("ADC map")
ax[0].axis('off')
ax[1].imshow(PIDS_1)
ax[1].set_title("ADC > 3")
ax[1].axis('off')
ax[2].imshow(PIDS_2)
ax[2].set_title("ADC < 0")
ax[2].axis('off')
fig, ax = plt.subplots(1,4, figsize = (40,10))
for i in range(PIDS_3.shape[2]):
    ax[i].imshow(PIDS_3[:, :, i])
    ax[i].set_title(f"TE = {i}")
    ax[i].axis('off')

fig, ax = plt.subplots(1,4, figsize = (40,10))
for i in range(PIDS_4.shape[2]):
    ax[i].imshow(PIDS_4[:, :, i])
    ax[i].set_title(f"b = {i}")
    ax[i].axis('off')

In [ ]:
from skimage import morphology
from skimage import segmentation
mask = hybrid_crop[:, :, sliceIndex, 0, 0]> 10000 #hybrid_crop[:, :, sliceIndex, 1, 1].max()*0.2
fig, ax = plt.subplots(1,3, figsize = (30,10))
mask = morphology.remove_small_objects(mask.astype(bool), min_size=50, connectivity=50)
mask = morphology.remove_small_holes(mask, area_threshold=50, connectivity=1)
ax[0].imshow(mask)
ax[0].set_title(f"TE = 1")
ax[0].axis('off')

ax[1].imshow(hybrid_crop[:,:, sliceIndex, 0, 0], cmap='gray', vmin=5000, vmax=50000)
ax[1].axis('off')
ax[2].imshow(adc_map, cmap='gray', vmin=0.3, vmax=3)
ax[2].axis('off')
PIDS = PIDS_1.astype(float) + PIDS_2.astype(float)
for i in range(4):
    PIDS += PIDS_3[:,:,i].astype(float)
    PIDS += PIDS_4[:,:,i].astype(float)
mask = mask*PIDS
mask = mask.astype(float)
mask[mask==0] = np.nan
ax[2].imshow(mask, cmap='autumn',alpha = 0.4)
ax[2].axis('off')